In [50]:
from collections import deque
from typing      import Iterator, List, Tuple
from helpers     import data

In [51]:
Instruction = Tuple[str, int] # ('jmp', 4)
Program = List[Instruction]

def parser(line: str) -> [str, int]:
    instr, n = line.split()
    return [instr, int(n)]

instructions = data(8, parser=parser)
instructions[:5]

[['jmp', 149], ['acc', -11], ['nop', 95], ['acc', -6], ['jmp', 196]]

**Part 1:** The accumulator starts at 0. What's its value right before an instruction is run for the second time? 

In [52]:
def run(fn=lambda instr, n, instr_index: None) -> Tuple[int, bool]:
    """Return accumulator and status before infinite loop starts.
    Status is False if infinite loop was detected. 
    """
    instr_run = set()
    acc = 0 
    instr_index = 0 
    while instr_index not in instr_run: 
        if instr_index >= len(instructions):
            return acc, True
        instr, n = instructions[instr_index]
        instr_run.add(instr_index)
        fn(instr, n, instr_index) # Run on each line 
        
        if instr == "jmp":
            instr_index += n
        elif instr == "acc":
            acc += n
            instr_index += 1
        elif instr == "nop":
            instr_index += 1
        else: 
            raise ValueError(f"Instruction {instr} wasn't expected.")
    return acc, False

In [53]:
run()

(1816, False)

**Norvig:** He does pretty much the same thing, but it's a lot more concise and his variable names are better (c'mon, I should have thought of `pc`!). 

In [55]:
def run(program: Program) -> Tuple[int, bool]: 
    """Return accumulator and terminates, before an infinite loop starts."""
    pc = acc = 0 
    executed = set() # Instruction addresses already run 
    while True: 
        if pc in executed: 
            return acc, False
        elif pc == len(program):
            return acc, True
        elif pc > len(program):
            return acc, False
        executed.add(pc)
        opcode, arg = program[pc]
        pc += 1
        if opcode == "acc":
            acc += arg
        elif opcode == "jmp":
            pc += arg - 1

In [56]:
run(instructions)

(1816, False)

**Part 2:** Change one "jmp" to "nop" or vice versa, so that program terminates (tries to execute instruction 1 after instruction set). An infite loop is caused by a "jmp" so find all "jmp" instructions called before infinite loop, and try switching them to "nop" one at a time. 

In [54]:
jmp_indices = deque()

def store_jmp(instr, n, instr_index): 
    """Call this on every instruction to note index of jmp instructions."""
    if instr == "jmp":
        jmp_indices.append(instr_index)

run(fn=store_jmp)

while jmp_indices: 
    jmp_index = jmp_indices.pop()
    # Convert instruction to nop and run 
    instructions[jmp_index][0] = "nop"
    acc, status = run()
    # Convert back
    instructions[jmp_index][0] = "jmp"
    if status:
        print(acc)
        break

1149


**Norvig:** Hm, I don't think my solution would always work. Sometimes, solving might require changing "nop" to "jmp". Norvig's solution is good because although he creates a new program for each possible instruction change, he uses a generator so there's no space overhead. I need to practice thinking of this. 

In [57]:
def altered_programs(program, other=dict(jmp="nop", nop="jmp")) -> Iterator[Program]:
    """Generate all possible altered programs by swapping opcodes as per other."""
    for i, (opcode, n) in enumerate(program): 
        if opcode in other: 
            yield [*program[:i], (other[opcode], n), *program[i + 1:]]

programs = altered_programs(instructions)

# Another generator to stop once we find a terminating program
next((acc for (acc, terminates) in map(run, programs) if terminates), None)

1149